# 使用预训练的VGG模型Fine-tune CNN

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

from six.moves import urllib
from six.moves import xrange  
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import h5py

import scipy.io
from scipy.misc import imread, imresize
import skimage.io
import skimage.transform

%matplotlib inline
cwd = os.getcwd()
print ("Package loaded")
print ("Current folder is %s" % (cwd) )

Package loaded
Current folder is C:\Users\ML\Desktop\DeepLearningCourseCodes-master\04_CNN_advances


In [2]:
# 下载预先训练好的vgg-19模型，为Matlab的.mat格式，之后会用scipy读取
# (注意此版本模型与此处http://www.vlfeat.org/matconvnet/pretrained/最新版本不同)
import os.path
if not os.path.isfile('./data/imagenet-vgg-verydeep-19.mat'):
    !wget -O data/imagenet-vgg-verydeep-19.mat http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat

# 载入图像，调节尺寸，生成数据集

In [17]:

base_dir = os.path.expanduser('F:\projects\data_tc')
path_training = os.path.join(base_dir, 'validation.h5')
path_validation = os.path.join(base_dir, 'training.h5')
path_test = os.path.join(base_dir, 'test_a.h5')

fid_training = h5py.File(path_training,'r')
fid_validation = h5py.File(path_validation,'r')
fid_test = h5py.File(path_test,'r')
    
train_s1 = fid_training['sen1']
train_s2 = fid_training['sen2']
train_label = fid_training['label']


validation_s1 = fid_validation['sen1']
validation_s2 = fid_validation['sen2']
validation_label = fid_validation['label']

# 定义VGG网络结构

In [4]:
def net(data_path, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )
    data = scipy.io.loadmat(data_path)
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]
    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current
    assert len(net) == len(layers)
    return net, mean_pixel
def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')
def preprocess(image, mean_pixel):
    return image - mean_pixel
def unprocess(image, mean_pixel):
    return image + mean_pixel
print ("VGG net ready")

VGG net ready


# 定义finetuning的结构

In [6]:
# Parameters
n_input  = 32 * 32 * 18
n_output = 17
n_channels = 18
stddev = 0.02
initial_learning_rate = 0.004 #初始学习率

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)
# Network
with tf.device("/cpu:0"):
    weights  = {
        'wd1': tf.Variable(tf.random_normal([n_input, 1024], stddev=stddev)),
        'wd2': tf.Variable(tf.random_normal([1024, n_output], stddev=stddev))
    }
    biases   = {
        'bd1': tf.Variable(tf.random_normal([1024], stddev=stddev)),
        'bd2': tf.Variable(tf.random_normal([n_output], stddev=stddev))
    }
    def conv_basic(_input, _w, _b, _keepratio):
        # Input
        _input_r = _input
        # Vectorize
        _dense1 = tf.reshape(_input_r, [-1, _w['wd1'].get_shape().as_list()[0]])
        # Fc1
        _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
        _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
        # Fc2
        _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
        # Return everything
        out = {'input_r': _input_r, 'dense1': _dense1,
            'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out }
        return out
    # Functions! 
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                               global_step=global_step,
                                               decay_steps=10,decay_rate=0.2)
    
    _pred = conv_basic(x, weights, biases, keepratio)['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
#     optm = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
    optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    _corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) 
    accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) 
    init = tf.initialize_all_variables()

print ("Network Ready to Go!")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Network Ready to Go!


In [7]:
sess = tf.Session()
sess.run(init)

In [18]:
training_epochs = 10
batch_size      = 512
display_step    = 1
for epoch in range(training_epochs):
#     if((epoch + 1) % 3 == 0):
#         add_global = global_step.assign_add(1)
#         _, rate = sess.run([add_global, learning_rate])
#         print("learning_rate : " + str(sess.run(learning_rate)))
    avg_cost = 0.
    total_batch = int(train_s1.shape[0]/batch_size)

    # Loop over all batches
    for i in range(total_batch):
        
        start_pos = i * batch_size
        end_pos = min((i + 1) * batch_size,train_s1.shape[0])
        if(i % 10 == 0):
            print("epoch : " +str(epoch) + ", batch_num : " + str(i)+ "/" + str(total_batch)  + "，pos : " + str(start_pos) + "/" + str(end_pos))

        train_s1_batch = np.asarray(train_s1[start_pos:end_pos, :, :, :])
        train_s2_batch = np.asarray(train_s2[start_pos:end_pos, :, :, :])
        cur_batch_size = train_s2_batch.shape[0]
        train_s1_batch = train_s1_batch.reshape((cur_batch_size, -1))
        train_s2_batch = train_s2_batch.reshape((cur_batch_size, -1))
        batch_xs = np.hstack([train_s1_batch, train_s2_batch])
        batch_ys = train_label[start_pos:end_pos]
        
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio:.5})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})/total_batch

    # Display logs per epoch step
    if epoch % display_step == 0: 
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})
        print ("Training accuracy: %.3f" % (train_acc))
        
        #电脑内存不够，随机取若干条数据做校验
        random_int=np.random.randint(10)
        start_pos = batch_size * random_int
        end_pos = batch_size * (random_int + 16)
        validation_s1_batch = np.asarray(validation_s1[start_pos:end_pos, :, :, :])
        validation_s2_batch = np.asarray(validation_s2[start_pos:end_pos, :, :, :])
        validation_batch_size = validation_s2_batch.shape[0]
        validation_s1_batch = validation_s1_batch.reshape((validation_batch_size, -1))
        validation_s2_batch = validation_s2_batch.reshape((validation_batch_size, -1))
        batch_xs_vali = np.hstack([validation_s1_batch, validation_s2_batch])
        batch_ys_vali = validation_label[start_pos:end_pos]

        feeds = {x: batch_xs_vali, y: batch_ys_vali, keepratio:1.}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("Test accuracy: %.3f" % (test_acc))
        
print ("OPTIMIZATION FINISHED")

#跑测试数据，并将结果写入csv文件中
test_s1 = fid_test['sen1']
test_s2 = fid_test['sen2']
test_num = test_s1.shape[0]
print("test_num : " + str(test_num))
test_total_batch = 1
test_batch_size = test_num // test_total_batch
print("test_batch_size : " + str(test_batch_size))

for i in range(test_total_batch):

    print("test_total_batch : " + str(test_total_batch) + ", batch_num : " + str(i))
    start_pos = test_batch_size * i
    end_pos = min(test_batch_size * (i + 1),test_num)
    test_s1_batch = np.asarray(test_s1[start_pos:end_pos, :, :, :])
    test_s2_batch = np.asarray(test_s2[start_pos:end_pos, :, :, :])
    test_batch_size = test_s2_batch.shape[0]
    test_s1_batch = test_s1_batch.reshape((test_batch_size, -1))
    test_s2_batch = test_s2_batch.reshape((test_batch_size, -1))
    batch_xs_test = np.hstack([test_s1_batch, test_s2_batch])
    batch_ys_test = validation_label[start_pos:end_pos]

    feeds = {x: batch_xs_test, y: batch_ys_test, keepratio:1.}
    test_pred_value = sess.run(_pred, feed_dict=feeds)
    argmax = tf.argmax(_pred,1)
    #将输出结果转化为01编码的数组
    test_argmax = sess.run(argmax, feed_dict=feeds)
    test_pred = np.zeros_like(test_pred_value,dtype=np.int)
    print(test_pred_value[:2])
    for i in range(test_pred.shape[0]):
        test_pred[i][test_argmax[i]]=1
    print(test_pred)

    np.savetxt("adelin100.csv", test_pred, delimiter=",",fmt='%d')
    
print("test finish")

epoch : 0, batch_num : 0/47，pos : 0/512
epoch : 0, batch_num : 10/47，pos : 5120/5632
epoch : 0, batch_num : 20/47，pos : 10240/10752
epoch : 0, batch_num : 30/47，pos : 15360/15872
epoch : 0, batch_num : 40/47，pos : 20480/20992
Epoch: 000/010 cost: 1.387878284
Training accuracy: 0.557
Test accuracy: 0.561
epoch : 1, batch_num : 0/47，pos : 0/512
epoch : 1, batch_num : 10/47，pos : 5120/5632
epoch : 1, batch_num : 20/47，pos : 10240/10752
epoch : 1, batch_num : 30/47，pos : 15360/15872
epoch : 1, batch_num : 40/47，pos : 20480/20992
Epoch: 001/010 cost: 1.261793547
Training accuracy: 0.576
Test accuracy: 0.563
epoch : 2, batch_num : 0/47，pos : 0/512
epoch : 2, batch_num : 10/47，pos : 5120/5632
epoch : 2, batch_num : 20/47，pos : 10240/10752
epoch : 2, batch_num : 30/47，pos : 15360/15872
epoch : 2, batch_num : 40/47，pos : 20480/20992
Epoch: 002/010 cost: 1.223818941
Training accuracy: 0.576
Test accuracy: 0.556
epoch : 3, batch_num : 0/47，pos : 0/512
epoch : 3, batch_num : 10/47，pos : 5120/5632
